In [ ]:
using_colab = True

In [ ]:
if using_colab:

    !wget 'https://github.com/Philliec459/Open-Source-Petrophysics/raw/main/NMR_Echo_Train_Simple_T2_Inversion_Example/MRIL_ECHOA_github.xlsx'
    !wget 'https://github.com/Philliec459/Open-Source-Petrophysics/raw/main/NMR_Echo_Train_Simple_T2_Inversion_Example/nmr_animation.gif'
    
    #https://github.com/Philliec459/Science-and-Technology-Society-Use-of-NASA-STELLA-Q2-Spectrometer/raw/main/STELLA_brief_ver2_backyard_grass_shoreline.ipynb
    #https://github.com/Philliec459/Science-and-Technology-Society-Use-of-NASA-STELLA-Q2-Spectrometer/raw/main/data_white_grass_shade_whiteshade.xlsx
    #https://github.com/Philliec459/Science-and-Technology-Society-Use-of-NASA-STELLA-Q2-Spectrometer/raw/main/data_white_FullSun.xlsx

    !pip install scipy


# **NMR Echo Train processing and creation of an NMR log in Real Time**

In this notebook we will process a MRIL-Prime ECHOA Echo Train with some stacking and then T2 inversion to create an NMR log.

![nmr_amimation](nmr_animation.gif)


## Import Python libraries:

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Sep  8 12:33:54 2020

@author: craig
"""
%matplotlib inline
#%matplotlib tk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize, curve_fit, least_squares



pd.set_option('display.max_rows', 500)

## Loading MRIL Data ECHOA:

The MRIL-Prime computed curves and ECHOA are available in an Excel file. 

In [ ]:
# Load the Excel file
file_path = "MRIL_ECHOA_github.xlsx"  # Update this if the file is in a different location
data = pd.read_excel(file_path, header=0)


# Check the shape of the data (optional)
print(f"Data shape: {data.shape}")

data.head()

In [ ]:
depth = data.iloc[:, 0]  # First column
#print(depth)


In [ ]:
echo_train = data.iloc[:, 7:]  # All columns except the first

echo_train.head()

## Create Curves for our work:

In [ ]:
depth = data['Depth']
mbvi_log = data['MBVIA']
mphi_log = data['MPHIA']
tphic_log = data['TPHIC']
te = data['ESPACEA']
echoa = echo_train
echoa = np.array(echoa)


print(f'Total Depth Interval: {depth.min()} - {depth.max()} m')
print(f'Interecho Spacing for EchoA (TE): {te.max()} ms')
TE = te.max()
print('TE =',TE)


## Note the TE used for this echo train

This TE needs to be used in the T2 inversion process.

#### **Trim first echo of echoa:**

The first echo is notoriously noisy and is usually dropped (at least it was in the past).

In the echo train plolts to follow, if you set drop_echo = 0 then you will see the first noisy echo in some of the plots. For this data we suggest dropping the first echo. 

In [ ]:
echoa.shape

In [ ]:
# Trim first echo of echoa
#drop_echo = 0 
drop_echo = 1

echoa = echoa[:, drop_echo:]

In [ ]:
echoa.shape

In [ ]:
print(len(echoa[1]))

In [ ]:
num_echoes = len(echoa[1])
print('num_echoes:',num_echoes)

---
---
## **Plot some basic data**

### Entire Well First:

In [ ]:
fig = plt.subplots(figsize=(5,5))

plt.plot(mbvi_log,depth ,c='blue',linewidth=1, label='BVI')
plt.plot(mphi_log,depth ,'r-',linewidth=1 , label='EFfective NMR Porosity')
plt.plot(tphic_log,depth ,'k-',linewidth=0.5, label='Total NMR Porosity')
plt.xlim(30,0)
plt.ylim(depth.max(), depth.min())
plt.xlabel('Porosity (pu)')
plt.ylabel('Depth (feet)')
plt.title('NMR Log using 100ms T2 Cutoff') 
plt.fill_betweenx(depth, tphic_log, 0,  color='gray', alpha=0.9, label='CBW')   
plt.fill_betweenx(depth, mphi_log, 0,  color='yellow', alpha=0.9, label='FFI')   
plt.fill_betweenx(depth, mbvi_log, 0,  color='blue'  , alpha=0.9, label='BVI')    
#plt.fill_betweenx(depth, tcmr, cmrp_3ms,  color='gray', alpha=0.9, label='CBW')   
plt.legend(loc='upper left')
plt.grid()

plt.show()

## Now our Zone of Interest (ZoI):

This can be used on other wells where the ZoI would be quite different than over the entire well. 

We just want this function available to the user.

In [ ]:
# What is the Zone of Interest (ZoI) that you want to evaluate? 
# Use the log created below to determine the ZoI
depth_min = 1001
depth_max = 1039


fig = plt.subplots(figsize=(5,5))

plt.plot(mbvi_log,depth ,c='blue',linewidth=1, label='BVI')
plt.plot(mphi_log,depth ,'r-',linewidth=1 , label='EFfective NMR Porosity')
plt.plot(tphic_log,depth ,'k-',linewidth=0.5, label='Total NMR Porosity')
plt.xlim(30,0)
plt.ylim(depth_max, depth_min)
plt.xlabel('Porosity (pu)')
plt.ylabel('Depth (feet)')
plt.title('NMR Log using 100ms T2 Cutoff') 
plt.fill_betweenx(depth, tphic_log, 0,  color='gray', alpha=0.9, label='CBW')   
plt.fill_betweenx(depth, mphi_log, 0,  color='yellow', alpha=0.9, label='FFI')   
plt.fill_betweenx(depth, mbvi_log, 0,  color='blue'  , alpha=0.9, label='BVI')    
#plt.fill_betweenx(depth, tcmr, cmrp_3ms,  color='gray', alpha=0.9, label='CBW')   
plt.legend(loc='upper left')
plt.grid()

plt.show()

## Stack a small section of ECHOA and plot the stacked echo trains.

### ECHOA Stacked:

In [ ]:
stack_size = 50  # Number of depth levels to stack

# Find indices within the depth range
indices = np.where((depth >= depth_min) & (depth <= depth_max))[0]
selected_depths = depth[indices]

# Loop through depth groups of 5 and plot the averaged echo train
for i in range(0, len(indices), stack_size):
    # Ensure we have a full stack of 5 levels
    if i + stack_size > len(indices):
        break
    
    # Get the current depth indices
    current_indices = indices[i:i + stack_size]
    current_depths = depth[current_indices]
    
    # Average the echo trains over these depth levels
    #averaged_echo_train = np.mean(echo_Corr[current_indices, :num_echoes], axis=0)
    averaged_echo_train = np.mean(echoa[current_indices, :num_echoes], axis=0)
    #averaged_echo_train = np.mean(echo_X[current_indices, :num_echoes], axis=0)

    print('mean of averaged echo train:', np.mean(averaged_echo_train))
    
    # Plot the averaged echo train
    plt.figure(figsize=(8, 4))
    plt.plot(
        np.arange(num_echoes),  # Echo indices
        averaged_echo_train,  # Averaged echo train
        label=f"Depths: {current_depths.min():.2f} to {current_depths.max():.2f} ft",
        color="red"
    )
    plt.title(f"Averaged Echo Train for Depths {current_depths.min():.2f}-{current_depths.max():.2f} ft")
    plt.xlabel("Echo Number")
    plt.ylabel("Amplitude (averaged over 5 levels)")
    plt.ylim(0,20)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()


## Create a new, stacked Echo Train **echo_Corr_stacked** over entire log section:

In [ ]:
# Parameters
stack_size = 3  # Number of levels above and below to include
#num_echoes = echoa.shape[1]  # Number of echoes
num_depths = len(depth)  # Total number of depth levels

# Initialize the stacked echo train array
echo_Corr_stacked = np.zeros_like(echoa)

# Iterate over each depth level
for i in range(num_depths):
    # Determine the range of depths to average
    start_idx = max(0, i - stack_size)  # Ensure we don't go above the first depth
    end_idx = min(num_depths, i + stack_size + 1)  # Ensure we don't go beyond the last depth
    
    # Average the echo values for each echo across the selected depth range
    echo_Corr_stacked[i, :] = np.mean(echoa[start_idx:end_idx, :], axis=0)

# echo_Corr_stacked now contains the stacked echo train for each depth


---
---
## **T2 Inversion to create MRIL Log**


    Bin Numbers: 1  2   3   4   5    6    7    8    9
    T2:          4  8  16  32  64   128  256  512 1024   

In [ ]:

'''
The Inversion will be performed over the Zone if Interest
'''



# T2 relaxation times for inversion
T2 = [2, 4, 8, 16, 32, 64, 128, 256, 512, 1024,2048]

def func(x, p1, p2, p3, p4, p5, p6, p7, p8, p9):
    return (
            p1 * np.exp(-x / 4) +  
            p2 * np.exp(-x / 8) +
            p3 * np.exp(-x / 16) +
            p4 * np.exp(-x / 32) +
            p5 * np.exp(-x / 64) +
            p6 * np.exp(-x / 128) +
            p7 * np.exp(-x / 256) +
            p8 * np.exp(-x / 512) +
            p9 * np.exp(-x / 1024)
           )


''' Calculate T2 Cutoff of 33ms '''
# Calculate BFV and FFI fractions for the 32ms bin
# Sandstone
log_33  = np.log10(33)
log_32  = np.log10(32)
log_64 = np.log10(64)
BVI_fraction_ss = (log_33 - log_32) / (log_64 - log_32)      # Log-based BVI fraction
FFI_fraction_ss = 1 - BVI_fraction_ss                        # Remaining portion for FFI



# Carbonate
log_100 = np.log10(100)
log_64  = np.log10(64)
log_128 = np.log10(128)
BVI_fraction_co3 = (log_100 - log_64) / (log_128 - log_64)     # Log-based BVI fraction
FFI_fraction_co3 = 1 - BVI_fraction_co3                        # Remaining portion for FFI





# Initialize arrays
deptharray = []
mphiarray = []    
mbviarray = []
mffiarray = []


# Parameters
#TE = 0.9  # Echo spacing (ms)
x = np.arange(num_echoes) * TE  # Time array for echoes (first 5000 echoes only)
xdata = x
method = 'least_squares'  # Choose the fitting method
alpha = 2  # Regularization term for least squares

'''
# Reduce to 5 samples per meter
#indices = indices[::2]  # Take every second depth level
indices = np.array(indices)

# Reduce to 5 samples per meter by taking every second index
reduced_indices = indices[::2]  # Take every second element from indices
'''

# Find the indices of the depth levels within the specified range
indices = np.where((depth >= depth_min) & (depth <= depth_max))[0]

#for i in reduced_indices:  # Process only the selected depth levels
for i in indices:  # Process only the selected depth levels

    # Use the stacked echo train for this depth level (first 5000 echoes)
    ydata = echo_Corr_stacked[i, :num_echoes]


    # Set initial parameters
    p = np.ones(9)

    # Define objective function for least squares
    def objective(p, x, y, alpha):
        y_pred = func(x, *p)
        error = y - y_pred
        return np.concatenate([error, np.sqrt(alpha) * p])

    # Perform T2 inversion
    result = least_squares(objective, p, args=(xdata, ydata, alpha), bounds=([0]*9, [20]*9))
    popt = result.x



    
    ''' Calculate BVI using Fixed T2 Cutoff '''
    # Sandstone
    #mbvi = (popt[0] + popt[1] + popt[2] + popt[3]*BVI_fraction_ss)    
    
    # Carbonate
    mbvi = (popt[0] + popt[1] + popt[2] + popt[3] + popt[4]*BVI_fraction_co3)    
    ''' End of Calculating BVI using Fixed T2 Cutoff '''


    
    ''' SBVI '''
    #SBVI_coef = 0.0618  # SBVI Rx Coef for SS
    SBVI_coef  = 0.0112  # SBVI Rx Coef for CO3

    c1 = 1/(SBVI_coef*4+1)
    c2 = 1/(SBVI_coef*8+1)
    c3 = 1/(SBVI_coef*16+1)
    c4 = 1/(SBVI_coef*32+1)
    c5 = 1/(SBVI_coef*64+1)
    c6 = 1/(SBVI_coef*128+1)
    c7 = 1/(SBVI_coef*256+1)
    c8 = 1/(SBVI_coef*512+1)
    c9 = 1/(SBVI_coef*1024+1)

    sbvi = c1*popt[0] + c2*popt[1] + c3*popt[2] + c4*popt[3] + c5*popt[4] + c6*popt[5] + c7*popt[6] + c8*popt[7] + c9*popt[8] 
    ''' End of SBVI '''

    mbvi = max(mbvi,sbvi)
    ''' End of BVI using Max of Fixed T2 Cutoff or SBVI for our final BVI'''


    
    mbviarray.append(mbvi)    
    mffiarray.append(np.sum(popt) - mbvi)
    mphiarray.append(np.sum(popt))
    deptharray.append(depth[i])

    # T2 Distribution for plotting
    #T2 =       2,    4,      8,     16,    32,      64,     128,    256,   512,    1024      2048
    optbins  = [0 ,popt[0], popt[1],popt[2],popt[3],popt[4],popt[5],popt[6],popt[7],popt[8],    0 ]


    # Plot results for each depth level
    fig, ax = plt.subplots(1, 3, figsize=(15, 4))
    
    # Echo train and fit (ax[0])
    ax[0].plot(xdata, ydata, c='green', label='Stacked Echo Train')
    ax[0].plot(xdata, func(xdata, *popt), c='red', label='Fit')
    ax[0].set_xlim(0, num_echoes * TE)  # Adjust based on desired echo time range
    ax[0].set_ylim(-5, 30)
    ax[0].set_title('Echo Train and Fit')
    ax[0].set_xlabel('Time (ms)')
    ax[0].set_ylabel('Amplitude')
    ax[0].legend()
    ax[0].grid()

    # T2 distribution (ax[1])
    ax[1].semilogx(T2, optbins, c='red', linewidth=3, label='T2 Distribution')
    ax[1].set_xlim(2, 2048)
    ax[1].set_ylim(0, 15)
    ax[1].set_title('T2 Distribution')
    ax[1].set_xlabel('T2 (ms)')

    
    ''' Sandstone Background Fill for BVI and FFI'''
    #ax[1].fill_betweenx([0, 15], 2,    33, color='blue', alpha=0.2, label='BVI')
    #ax[1].fill_betweenx([0, 15], 33, 2048, color='yellow', alpha=0.5, label='FFI')
    ''' Carbonate Background Fill for BVI and FFI'''
    ax[1].fill_betweenx([0, 15], 2,    100, color='blue'  , alpha=0.2, label='BVI')
    ax[1].fill_betweenx([0, 15], 100, 2048, color='yellow', alpha=0.5, label='FFI')

    
    ax[1].legend()
    ax[1].grid()

    # NMR log (ax[2])
    ax[2].plot(mphiarray, deptharray, c='red', label='NMR Porosity')
    ax[2].plot(mbviarray, deptharray, c='blue', label='BVI')
    ax[2].set_xlim(30, 0)
    ax[2].set_ylim(depth_max, depth_min)
    #ax[2].invert_yaxis()
    ax[2].set_title('MRIL Log')
    ax[2].set_xlabel('Porosity (pu)')
    ax[2].fill_betweenx(deptharray, mphiarray, 0, color='yellow', alpha=0.9, label='FFI')   
    ax[2].fill_betweenx(deptharray, mbviarray, 0, color='blue', alpha=0.9, label='BVI')    
    ax[2].legend()
    ax[2].grid()

    plt.tight_layout()
    plt.show()


## Plot of Original Data:

In [ ]:


fig = plt.subplots(figsize=(5,5))

plt.plot(mbvi_log,depth ,c='blue',linewidth=1, label='BVI')
plt.plot(mphi_log,depth ,'r-',linewidth=1 , label='EFfective NMR Porosity')
plt.plot(tphic_log,depth ,'k-',linewidth=0.5, label='Total NMR Porosity')
plt.xlim(30,0)
plt.ylim(depth_max, depth_min)
plt.xlabel('Porosity (pu)')
plt.ylabel('Depth (feet)')
plt.title('NMR Log using Max (100ms T2 Cutoff, SBVI)') 
plt.fill_betweenx(depth, tphic_log, 0,  color='gray', alpha=0.9, label='CBW')   
plt.fill_betweenx(depth, mphi_log, 0,  color='yellow', alpha=0.9, label='FFI')   
plt.fill_betweenx(depth, mbvi_log, 0,  color='blue'  , alpha=0.9, label='BVI')    
#plt.fill_betweenx(depth, tcmr, cmrp_3ms,  color='gray', alpha=0.9, label='CBW')   
plt.legend(loc='upper left')
plt.grid()

plt.show()